In [85]:
## Helper Functions & Query Set up

import requests
import json
import time
from datetime import timedelta, datetime, timezone
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)   
pd.set_option('display.max_colwidth', None) 

current_utc_time = datetime.now(timezone.utc)
queryTime = current_utc_time - timedelta(hours=4)
#queryTime = '2025-08-07 5:00:00'
queryEndTime = queryTime + timedelta(hours = 2)
#queryEndTime = '2025-08-08 5:00:00.000000+00:00'

query = f"""
            WITH TeleopEvents AS (
    SELECT 
        bot_id,
        datetime AS teleop_time,
        subevent
    FROM SYMBOTIC_KAFKA_DB.LOG_TABLES.DIAGNOSTICMESSAGE
    WHERE datetime >= '{queryTime}'
    --AND datetime <= '{queryEndTime}'
    --WHERE datetime >= '2025-08-03'
        AND site_id in('walmart-tx-newbraunfels-site1')--, 'walmart-oh-grovecity-site1')
        AND (
            (subevent = 'ENTER_TELEOPS' AND actionecode = 0)
            OR subevent = 'EXIT_TELEOPS' 
        )
),
PairedTeleopTimes AS (
    SELECT 
        e1.bot_id,
        e1.teleop_time AS enter_teleop_time,
        MIN(e2.teleop_time) AS exit_teleop_time
    FROM TeleopEvents e1
    JOIN TeleopEvents e2 
        ON e1.bot_id = e2.bot_id
        AND e1.subevent = 'ENTER_TELEOPS'
        AND e2.subevent = 'EXIT_TELEOPS'
        AND e2.teleop_time > e1.teleop_time
    GROUP BY e1.bot_id, e1.teleop_time
),
LocationInfo AS (
    SELECT 
        datetime AS UTC_datetime,
        DATEADD('hour', -5, datetime) AS local_datetime, 
        TO_DATE(DATEADD('hour',-5, datetime)) as productionDay,
        bot_id, 
        site_id,
        event, 
        subevent,
        location_level,
        location_x,
        location_y,
        location_yaw,
        velocity_linear,
        velocity_rotational,

        CASE 
            WHEN location_yaw >= 0 THEN location_yaw
            ELSE 2 * PI() + location_yaw
        END AS theta,
        ROUND(COS(CASE 
            WHEN location_yaw >= 0 THEN location_yaw
            ELSE 2 * PI() + location_yaw
        END), 0) AS x_theta,
        ROUND(SIN(CASE 
            WHEN location_yaw >= 0 THEN location_yaw
            ELSE 2 * PI() + location_yaw
        END), 0) AS y_theta,

        CASE 
            WHEN ABS(velocity_rotational) > 0.5 THEN 'Turning'
            ELSE 'DrivingStraight'
        END AS drivingType,
        CASE 
            WHEN velocity_linear > 0 THEN 'CasterLeading'
            ELSE 'TractionLeading'
        END AS drivingOrientation, 

        CASE 
            WHEN velocity_linear < 0 THEN ROUND(COS(CASE WHEN location_yaw >= 0 THEN location_yaw ELSE 2 * PI() + location_yaw END), 0) * -1
            ELSE ROUND(COS(CASE WHEN location_yaw >= 0 THEN location_yaw ELSE 2 * PI() + location_yaw END), 0)
        END AS x_drv_direction,

        CASE 
            WHEN velocity_linear < 0 THEN ROUND(SIN(CASE WHEN location_yaw >= 0 THEN location_yaw ELSE 2 * PI() + location_yaw END), 0) * -1
            ELSE ROUND(SIN(CASE WHEN location_yaw >= 0 THEN location_yaw ELSE 2 * PI() + location_yaw END), 0)
        END AS y_drv_direction,

        CASE
            WHEN ABS(velocity_rotational) > 0.5 THEN 
                CASE 
                    WHEN velocity_rotational > 0 THEN 'CounterClockwise'
                    ELSE 'Clockwise'
                END 
            ELSE 
                CASE 
                    WHEN ROUND(SIN(CASE WHEN location_yaw >= 0 THEN location_yaw ELSE 2 * PI() + location_yaw END), 0) = 1 THEN 'Driving North'
                    WHEN ROUND(SIN(CASE WHEN location_yaw >= 0 THEN location_yaw ELSE 2 * PI() + location_yaw END), 0) = -1 THEN 'Driving South'
                    WHEN ROUND(COS(CASE WHEN location_yaw >= 0 THEN location_yaw ELSE 2 * PI() + location_yaw END), 0) = 1 THEN 'Driving East'
                    WHEN ROUND(COS(CASE WHEN location_yaw >= 0 THEN location_yaw ELSE 2 * PI() + location_yaw END), 0) = -1 THEN 'Driving West'
                END 
        END AS driving_direction,

        COS(CASE WHEN location_yaw >= 0 THEN location_yaw ELSE 2 * PI() + location_yaw END) * 1.28 + location_x AS x_front,
        SIN(CASE WHEN location_yaw >= 0 THEN location_yaw ELSE 2 * PI() + location_yaw END) * 1.28 + location_y AS y_front,
        COS(CASE WHEN location_yaw >= 0 THEN location_yaw ELSE 2 * PI() + location_yaw END) * (-0.18) + location_x AS x_rear,
        SIN(CASE WHEN location_yaw >= 0 THEN location_yaw ELSE 2 * PI() + location_yaw END) * (-0.18) + location_y AS y_rear,

        CASE 
            WHEN velocity_linear > 0 THEN COS(CASE WHEN location_yaw >= 0 THEN location_yaw ELSE 2 * PI() + location_yaw END) * 1.28 + location_x
            ELSE COS(CASE WHEN location_yaw >= 0 THEN location_yaw ELSE 2 * PI() + location_yaw END) * (-0.18) + location_x
        END AS x_leading_edge,

        CASE 
            WHEN velocity_linear > 0 THEN SIN(CASE WHEN location_yaw >= 0 THEN location_yaw ELSE 2 * PI() + location_yaw END) * 1.28 + location_y
            ELSE SIN(CASE WHEN location_yaw >= 0 THEN location_yaw ELSE 2 * PI() + location_yaw END) * (-0.18) + location_y
        END AS y_leading_edge,

        CASE 
            WHEN (location_y <7.7 AND location_y > 7.2 AND velocity_linear > 0 AND ABS(velocity_rotational) <= 0.5) THEN 'StructureEdgeCollision'
            WHEN (location_y <0.3 AND location_y > 0.1 AND velocity_linear < 0 AND ABS(velocity_rotational) <= 0.5) THEN 'StructureEdgeCollision'
            WHEN (location_y > 7.7 AND location_y < 9.3 AND velocity_linear > 0 AND ABS(velocity_rotational) <= 0.5) THEN 'EnteringAisle'
            WHEN (location_y > -1.7 AND location_y < 0.1 AND velocity_linear < 0 AND ABS(velocity_rotational) <= 0.5) THEN 'EnteringDriveway'
            WHEN (location_y > 6.3 AND location_y < 9.3 AND velocity_linear < 0 AND ABS(velocity_rotational) <= 0.5) THEN 'ExitingAisle'
            WHEN (location_y > -1.7 AND location_y < 1.2 AND velocity_linear > 0 AND ABS(velocity_rotational) <= 0.5) THEN 'ExitingDriveway'
            ELSE NULL 
        END AS structure_impact,
        filename, 
        swversion,
        descriptor

    FROM SYMBOTIC_KAFKA_DB.LOG_TABLES.DIAGNOSTICMESSAGE
    WHERE site_id = 'walmart-tx-newbraunfels-site1'
    AND (event = 'collisionDetected')-- AND subevent = 'highMagnitudeCollision')
    AND datetime >= '{queryTime}'
    --AND datetime <= '{queryEndTime}'
    --AND datetime >= '2025-08-04'
)
SELECT 
    li.UTC_datetime,
    li.local_datetime, 
    li.productionDay,
    li.bot_id, 
    li.site_id,
    li.event, 
    li.subevent,
    li.location_level,
    li.location_x,
    li.location_y,
    li.location_yaw,
    li.x_leading_edge,
    li.y_leading_edge,
    li.velocity_linear,
    li.velocity_rotational,
    li.drivingtype, 
    li.drivingorientation, 
    li.driving_direction,
    li.structure_impact, 
    li.filename, 
    li.swversion,
    li.descriptor, 
    CASE 
        WHEN EXISTS (
            SELECT 1 
            FROM PairedTeleopTimes pt 
            WHERE pt.bot_id = li.bot_id 
              AND li.UTC_datetime BETWEEN pt.enter_teleop_time AND pt.exit_teleop_time
        ) THEN 'During Teleops'
        ELSE NULL
    END AS teleop_status
FROM LocationInfo li
ORDER BY UTC_datetime ASC;"""

In [ ]:
## Connect to Snowflake 


import snowflake.connector
import os
import pandas as pd

# ENTER YOUR SNOWFLAKE CREDENTIALS
# If you don't have them here, or as environment variables, you'll be prompted on run.
SNOWFLAKE_USER = "nsegale" 
SNOWFLAKE_PASSWORD = "Symb0t1c142831"

if SNOWFLAKE_USER == "":
    SNOWFLAKE_USER = os.environ.get("SNOWFLAKE_USER")
    if not SNOWFLAKE_USER or SNOWFLAKE_USER == "":
        SNOWFLAKE_USER = input("Enter your Snowflake username: ")
if SNOWFLAKE_PASSWORD == "":
    SNOWFLAKE_PASSWORD = os.environ.get("SNOWFLAKE_PASSWORD")
    if not SNOWFLAKE_PASSWORD or SNOWFLAKE_PASSWORD == "":
        SNOWFLAKE_PASSWORD = input("Enter your Snowflake password: ")

conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account='fb30115.east-us-2.azure',
    warehouse="BOTANALYTICS_WH"
)

# Create a cursor object
cur = conn.cursor()



c:\Users\nsegale\AppData\Local\Programs\Python\Python312\Lib\site-packages\snowflake\connector\config_manager.py:351: UserWarning: Bad owner or permissions on C:\Users\nsegale\.snowflake\connections.toml
  warn(f"Bad owner or permissions on {str(filep)}{chmod_message}")


In [86]:
##Run query 

from datetime import timedelta
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)   
pd.set_option('display.max_colwidth', None) 


# df = pd.read_csv('testerResultsNBF.csv')
 # where site_id in ('walmart-tx-newbraunfels-site1') 
cur.execute(query)

rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]

df = pd.DataFrame(rows, columns=cols)

In [76]:
## Grouping of resulting dataframe 

high_mag = df[df['SUBEVENT'] == 'highMagnitudeCollision'].copy()

print(type(df['LOCATION_Y']))
df['LOCATION_Y'] = pd.to_numeric(df['LOCATION_Y'], errors='coerce')
df['LOCATION_X'] = pd.to_numeric(df['LOCATION_X'], errors='coerce')

# df = df[(df['LOCATION_Y'] >= 0) & (df['LOCATION_Y'] <= 8)]
df['DATETIME'] = pd.to_datetime(df['UTC_DATETIME'], errors='coerce')
df['DAY'] = df['DATETIME'].dt.strftime('%B %#d') 
# df['LOCATION_LEVEL'] = 7

df = df.sort_values(by='DATETIME')

df['GROUP_ID'] = np.nan  

def is_close(instance1, instance2):
    if(instance1['SUBEVENT'] == 'highMagnitudeCollision' and instance2['SUBEVENT'] == 'highMagnitudeCollision'):
        return (
        abs(instance1['DATETIME'] - instance2['DATETIME']) <= timedelta(seconds=300) and
        abs(instance1['LOCATION_X'] - instance2['LOCATION_X']) <= 5.0 and
        abs(instance1['LOCATION_Y'] - instance2['LOCATION_Y']) <= 5.0 and
        instance1['BOT_ID'] != instance2['BOT_ID']
    )
    elif(instance1['SUBEVENT'] == 'highMagnitudeCollision' or instance2['SUBEVENT'] == 'highMagnitudeCollision'):
        return (
        abs(instance1['DATETIME'] - instance2['DATETIME']) <= timedelta(seconds=120) and
        abs(instance1['LOCATION_X'] - instance2['LOCATION_X']) <= 2.5 and
        abs(instance1['LOCATION_Y'] - instance2['LOCATION_Y']) <= 2.5 and
        instance1['BOT_ID'] != instance2['BOT_ID']
    )
    else:
        return (
        abs(instance1['DATETIME'] - instance2['DATETIME']) <= timedelta(seconds=60) and
        abs(instance1['LOCATION_X'] - instance2['LOCATION_X']) <= 2.5 and
        abs(instance1['LOCATION_Y'] - instance2['LOCATION_Y']) <= 2.5 and
        instance1['BOT_ID'] != instance2['BOT_ID']
        )

group_id = 1

for day in df['DAY'].unique():
    for level in df['LOCATION_LEVEL'].dropna().unique():
        dfFiltered = df[(df['DAY'] == day) & (df['LOCATION_LEVEL'] == level)].copy() #@Nicoel add site id and time diff within 30 
        dfFiltered.reset_index(inplace=True)

        seen = set()

        for idx, row in dfFiltered.iterrows():
            if idx in seen:
                continue

            group = [idx]
            queue = [idx]
            seen.add(idx)

            while queue:
                current_idx = queue.pop()
                current_row = dfFiltered.loc[current_idx]

                for other_idx, other_row in dfFiltered.iterrows():
                    if other_idx in seen:
                        continue
                    if is_close(current_row, other_row):
                        group.append(other_idx)
                        queue.append(other_idx)
                        seen.add(other_idx)
 
            if len(set(dfFiltered.loc[group, 'BOT_ID'])) > 1:
                df.loc[dfFiltered.loc[group, 'index'], 'GROUP_ID'] = group_id
                group_id += 1
        
high_mag = df[df['SUBEVENT'] == 'highMagnitudeCollision'].copy()

result_df = df[df['GROUP_ID'].notna()].copy()

high_mag['BOT_ID'] = high_mag['BOT_ID'].astype(str).str.strip()
result_df['BOT_ID'] = result_df['BOT_ID'].astype(str).str.strip()


merged = high_mag.merge(
    result_df[['BOT_ID', 'DATETIME']].drop_duplicates(),
    on=['BOT_ID', 'DATETIME'],
    how='left',
    indicator=True
)

for i in range(1, group_id + 1):
    dfGroup = result_df[result_df['GROUP_ID'] == i]
    if 'highMagnitudeCollision' not in dfGroup['SUBEVENT'].values:
            result_df = result_df[result_df['GROUP_ID'] != i]


result_df = result_df.sort_values(
    by=['GROUP_ID', 'SUBEVENT'],
    key=lambda col: (
        col if col.name != 'SUBEVENT' else col != 'highMagnitudeCollision'  
    )
)
solo_high_mag = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge']).reset_index(drop=True)
result_df = result_df.drop_duplicates(subset=['BOT_ID', 'GROUP_ID'])
## groupings
#display(result_df)

## solo high mags
#display(solo_high_mag)

<class 'pandas.core.series.Series'>


In [78]:
## Getting descriptor info 
import re

group_id = max(result_df['GROUP_ID'])+1

for idx in solo_high_mag.index:
    solo_high_mag.at[idx, 'GROUP_ID'] = group_id
    group_id += 1

result = pd.concat([result_df, solo_high_mag], ignore_index=True)

def extract_descriptor_values(descriptor):
    if pd.isna(descriptor):
        return [np.nan] * 4
    
    match = re.search(
        r"Current velocity: ([\-\d\.]+) m/s, Velocity Before Collision: ([\-\d\.]+) m/s, Collision Probability: ([\d\.]+)",
        descriptor
    )
    
    if match:
        current_velocity = float(match.group(1))
        velocity_before = float(match.group(2))
        collision_probability = float(match.group(3))
        delta_velocity = velocity_before - current_velocity
        return [current_velocity, velocity_before, delta_velocity, collision_probability]
    else:
        return [np.nan] * 4

# Apply the extraction
result[['CURRENT_VELOCITY', 'VELOCITY_BEFORE', 'DELTA_VELOCITY', 'COLLISION_PROBABILITY']] = result['DESCRIPTOR'].apply(lambda d: pd.Series(extract_descriptor_values(d)))


In [79]:
## Creating Instance IDS

import pandas as pd
# Ensure datetime column is parsed as datetime
result['LOCAL_DATETIME'] = pd.to_datetime(result['LOCAL_DATETIME'])

def create_instance_id(group):
    bot_ids = sorted(group['BOT_ID'].unique())  # unique bot IDs in group
    bot_str = ' & '.join(bot_ids)               # format: BOTID1 & BOTID2
    earliest_time = group['LOCAL_DATETIME'].min()
    time_str = earliest_time.strftime('%d%m%y%H%M')  # format: DDMMYYHHMM
    return bot_str + ' - ' + time_str

# Create a mapping from GROUP_ID to instance_ID
instance_id_map = result.groupby('GROUP_ID').apply(create_instance_id).to_dict()

# Map instance_ID to each row based on GROUP_ID
result['INSTANCE_ID'] = result['GROUP_ID'].map(instance_id_map)

#display(result)

C:\Users\nsegale\AppData\Local\Temp\ipykernel_26580\2229050197.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  instance_id_map = result.groupby('GROUP_ID').apply(create_instance_id).to_dict()


In [80]:
## Final Result for Sheet 
# Define your desired column order
desired_columns = [
    'UTC_DATETIME',
    'LOCAL_DATETIME',
    'BOT_ID',
    'INSTANCE_ID',
    'FILENAME',
    'LOCATION_LEVEL',
    'SUBEVENT',
    'TELEOP_STATUS',
    'CURRENT_VELOCITY',
    'VELOCITY_BEFORE',
    'DELTA_VELOCITY',
    'COLLISION_PROBABILITY',
    'SWVERSION',
    'SITE_ID',              # Make sure the case matches exactly
    'PRODUCTIONDAY',        # Assuming this is the correct name (vs. PRODUCTION_DAY)
    'LOCATION_X',
    'LOCATION_Y',
    'LOCATION_YAW',
    'X_LEADING_EDGE',
    'Y_LEADING_EDGE',
    'VELOCITY_LINEAR',
    'VELOCITY_ROTATIONAL',
    'DRIVINGTYPE',
    'DRIVINGORIENTATION',
    'DRIVING_DIRECTION',
    'STRUCTURE_IMPACT'
]

# Reorder the DataFrame
final_result = result[desired_columns]

display(final_result)

,UTC_DATETIME,LOCAL_DATETIME,BOT_ID,INSTANCE_ID,FILENAME,LOCATION_LEVEL,SUBEVENT,TELEOP_STATUS,CURRENT_VELOCITY,VELOCITY_BEFORE,DELTA_VELOCITY,COLLISION_PROBABILITY,SWVERSION,SITE_ID,PRODUCTIONDAY,LOCATION_X,LOCATION_Y,LOCATION_YAW,X_LEADING_EDGE,Y_LEADING_EDGE,VELOCITY_LINEAR,VELOCITY_ROTATIONAL,DRIVINGTYPE,DRIVINGORIENTATION,DRIVING_DIRECTION,STRUCTURE_IMPACT
0,2025-07-13 10:04:15.274,2025-07-13 05:04:15.274,8439,8439 & 9323 - 1307250504,bot_8439_20250713_100435_UTC.ion,2,highMagnitudeCollision,None,-0.356165,-0.986833,-0.630668,0.841784,3.169.0-nbf,walmart-tx-newbraunfels-site1,2025-07-13,-134.988113,-0.524316,1.573699832,-134.987591,-0.704315,-0.9868325591,0,DrivingStraight,TractionLeading,Driving North,EnteringDriveway
1,2025-07-13 10:04:15.591,2025-07-13 05:04:15.591,9323,8439 & 9323 - 1307250504,bot_9323_20250713_100435_UTC.ion,2,highMagnitudeCollision,None,0.392963,0.000000,-0.392963,0.564502,3.169.0-nbf,walmart-tx-newbraunfels-site1,2025-07-13,-133.975006,-1.029582,-1.5707963705,-133.975006,-0.849582,1.199e-07,0,DrivingStraight,TractionLeading,Driving South,None
2,2025-07-16 16:33:24.545,2025-07-16 11:33:24.545,31917,31763 & 31917 - 1607251131,bot_31917_20250716_163344_UTC.ion,8,highMagnitudeCollision,None,1.078006,1.609854,0.531848,0.705994,3.169.1-nbf,walmart-tx-newbraunfels-site1,2025-07-16,-52.597000,-3.870946,1.5707963705,-52.597000,-2.590946,1.6098539829,0,DrivingStraight,CasterLeading,Driving North,None
3,2025-07-16 16:31:44.890,2025-07-16 11:31:44.890,31763,31763 & 31917 - 1607251131,bot_31763_20250716_163205_UTC.ion,8,lowMagnitudeCollision,None,-1.448779,-1.861689,-0.412910,0.969212,3.169.1-nbf,walmart-tx-newbraunfels-site1,2025-07-16,-51.585999,-2.688587,-1.5707963705,-51.585999,-2.508587,-1.8616887331,0,DrivingStraight,TractionLeading,Driving South,None
4,2025-07-19 01:50:28.793,2025-07-18 20:50:28.793,31009,31009 & 32800 - 1807252050,bot_31009_20250719_015048_UTC.ion,8,highMagnitudeCollision,None,-0.133734,0.678973,0.812707,0.611776,3.170.0-nbf,walmart-tx-newbraunfels-site1,2025-07-18,-55.866844,1.196921,3.1309494972,-57.146772,1.210543,0.6789729595,0,DrivingStraight,CasterLeading,Driving West,ExitingDriveway
5,2025-07-19 01:50:29.210,2025-07-18 20:50:29.210,32800,31009 & 32800 - 1807252050,bot_32800_20250719_015052_UTC.ion,8,highMagnitudeCollision,None,-0.176467,0.501771,0.678238,0.926967,3.170.0-nbf,walmart-tx-newbraunfels-site1,2025-07-18,-58.937622,1.245369,0.3997294903,-57.758529,1.743506,0.501771152,0,DrivingStraight,CasterLeading,Driving East,None
6,2025-07-20 00:31:52.708,2025-07-19 19:31:52.708,31347,31347 & 31465 - 1907251931,bot_31347_20250720_003219_UTC.ion,3,highMagnitudeCollision,None,0.580999,2.005984,1.424985,0.975011,3.171.0-nbf,walmart-tx-newbraunfels-site1,2025-07-19,-114.514336,5.152023,3.1179394722,-115.793978,5.182297,2.0059840679,0,DrivingStraight,CasterLeading,Driving West,None
7,2025-07-20 00:31:52.802,2025-07-19 19:31:52.802,31465,31347 & 31465 - 1907251931,bot_31465_20250720_003212_UTC.ion,3,highImuValuesDetected,None,NaN,NaN,NaN,NaN,3.166.0-nbf,walmart-tx-newbraunfels-site1,2025-07-19,-116.653839,6.288772,-0.0209366139,-116.833800,6.292540,-1.2569954395,0.3144794106,DrivingStraight,TractionLeading,Driving East,None
8,2025-07-20 08:07:44.995,2025-07-20 03:07:44.995,31487,31487 & 31911 - 2007250307,bot_31487_20250720_080805_UTC.ion,8,highMagnitudeCollision,None,2.344548,3.832703,1.488155,0.945917,3.171.0-nbf,walmart-tx-newbraunfels-site1,2025-07-20,-50.844997,2.336778,-0.0928219408,-49.570508,2.218137,3.8327026367,0,DrivingStraight,CasterLeading,Driving East,None
9,2025-07-20 08:07:45.335,2025-07-20 03:07:45.335,31911,31487 & 31911 - 2007250307,bot_31911_20250720_080840_UTC.ion,8,highMagnitudeCollision,During Teleops,-0.279157,0.266893,0.546050,0.997097,3.171.0-nbf,walmart-tx-newbraunfels-site1,2025-07-20,-49.505226,3.489069,-1.5972802639,-49.539122,2.209517,0.2668934166,0,DrivingStraight,CasterLeading,Driving South,None


In [ ]:
key_cols = ['FILENAME', 'INSTANCE_ID', 'BOT_ID', 'UTC_DATETIME']

if 'pending_bart' in locals():
    # Create a merged key to compare
    pending_keys = pending_bart[key_cols].drop_duplicates()
    final_clean = final_result.merge(pending_keys, on=key_cols, how='left', indicator=True)
    final_clean = final_clean[final_clean['_merge'] == 'left_only'].drop(columns=['_merge'])

    pending_bart = pd.concat([pending_bart, final_clean], ignore_index=True)
else:
    pending_bart = final_result.copy()

#pending_bart

,UTC_DATETIME,LOCAL_DATETIME,BOT_ID,INSTANCE_ID,FILENAME,LOCATION_LEVEL,SUBEVENT,TELEOP_STATUS,CURRENT_VELOCITY,VELOCITY_BEFORE,DELTA_VELOCITY,COLLISION_PROBABILITY,SWVERSION,SITE_ID,PRODUCTIONDAY,LOCATION_X,LOCATION_Y,LOCATION_YAW,X_LEADING_EDGE,Y_LEADING_EDGE,VELOCITY_LINEAR,VELOCITY_ROTATIONAL,DRIVINGTYPE,DRIVINGORIENTATION,DRIVING_DIRECTION,STRUCTURE_IMPACT,BART_ID
0,2025-08-08 10:02:27.028,2025-08-08 05:02:27.028,32557,31888 & 32557 - 0808250502,bot_32557_20250808_100255_UTC.ion,9,highMagnitudeCollision,None,1.175310,-0.000469,-1.175779,0.851228,3.172.1-nbf,walmart-tx-newbraunfels-site1,2025-08-08,75.564369,6.315330,-0.0088048391,75.384376,6.316914,-0.0004686709,0,DrivingStraight,TractionLeading,Driving East,ExitingAisle,2135678.0
1,2025-08-08 10:02:27.136,2025-08-08 05:02:27.136,31888,31888 & 32557 - 0808250502,bot_31888_20250808_100247_UTC.ion,9,highMagnitudeCollision,None,0.769101,2.390391,1.621290,0.996078,3.172.1-nbf,walmart-tx-newbraunfels-site1,2025-08-08,74.377853,5.178880,-0.0691513792,75.654794,5.090437,2.3903906345,0,DrivingStraight,CasterLeading,Driving East,None,2135679.0
2,2025-08-08 10:32:49.498,2025-08-08 05:32:49.498,31786,31786 & 32009 - 0808250532,bot_31786_20250808_103309_UTC.ion,9,highMagnitudeCollision,None,0.936021,2.037962,1.101941,0.768140,3.172.1-nbf,walmart-tx-newbraunfels-site1,2025-08-08,77.014442,1.199653,0.0163648892,78.294271,1.220599,2.0379621983,0,DrivingStraight,CasterLeading,Driving East,ExitingDriveway,2135680.0
3,2025-08-08 10:32:49.558,2025-08-08 05:32:49.558,32009,31786 & 32009 - 0808250532,bot_32009_20250808_103356_UTC.ion,9,highMagnitudeCollision,None,1.002642,0.026500,-0.976142,0.837055,3.172.1-nbf,walmart-tx-newbraunfels-site1,2025-08-08,78.205048,2.404178,0.3788210154,79.394297,2.877555,0.0265003592,0,DrivingStraight,CasterLeading,Driving East,None,2135681.0
4,2025-08-08 10:56:20.477,2025-08-08 05:56:20.477,32745,31066 & 32745 - 0808250555,bot_32745_20250808_105726_UTC.ion,9,highMagnitudeCollision,None,0.127776,0.451885,0.324109,0.501299,3.172.1-nbf,walmart-tx-newbraunfels-site1,2025-08-08,74.549965,2.652800,1.5701420307,74.550802,3.932800,0.451884836,0,DrivingStraight,CasterLeading,Driving North,None,2135682.0
5,2025-08-08 10:55:20.707,2025-08-08 05:55:20.707,31066,31066 & 32745 - 0808250555,bot_31066_20250808_105714_UTC.ion,9,lowMagnitudeCollision,None,0.603695,0.717229,0.113534,0.777094,3.172.1-nbf,walmart-tx-newbraunfels-site1,2025-08-08,75.072639,3.300666,3.129127264,73.792739,3.316622,0.7172291875,0,DrivingStraight,CasterLeading,Driving West,None,2135683.0
6,2025-08-08 14:41:39.606,2025-08-08 09:41:39.606,24296,24296 & 8345 & 9336 - 0808250941,bot_24296_20250808_144200_UTC.ion,2,highMagnitudeCollision,None,1.930018,4.008336,2.078318,0.998503,3.172.1-nbf,walmart-tx-newbraunfels-site1,2025-08-08,9.271585,5.171690,-0.0122456271,10.551489,5.156016,4.0083360672,0,DrivingStraight,CasterLeading,Driving East,None,2135749.0
7,2025-08-08 14:42:59.323,2025-08-08 09:42:59.323,8345,24296 & 8345 & 9336 - 0808250941,bot_8345_20250808_144346_UTC.ion,2,highMagnitudeCollision,None,0.460611,0.988848,0.528237,0.876499,3.172.1-nbf,walmart-tx-newbraunfels-site1,2025-08-08,12.431705,7.016728,-1.5396043062,12.471624,5.737351,0.9888476133,0,DrivingStraight,CasterLeading,Driving South,None,2135751.0
8,2025-08-08 14:43:39.241,2025-08-08 09:43:39.241,9336,24296 & 8345 & 9336 - 0808250941,bot_9336_20250808_144359_UTC.ion,2,highMagnitudeCollision,None,0.036611,0.560702,0.524091,0.753009,3.172.1-nbf,walmart-tx-newbraunfels-site1,2025-08-08,7.955376,6.222530,-0.5122499466,9.071080,5.595152,0.5607019663,0,DrivingStraight,CasterLeading,Driving East,None,2135753.0
9,2025-07-13 10:04:15.274,2025-07-13 05:04:15.274,8439,8439 & 9323 - 1307250504,bot_8439_20250713_100435_UTC.ion,2,highMagnitudeCollision,None,-0.356165,-0.986833,-0.630668,0.841784,3.169.0-nbf,walmart-tx-newbraunfels-site1,2025-07-13,-134.988113,-0.524316,1.573699832,-134.987591,-0.704315,-0.9868325591,0,DrivingStraight,TractionLeading,Driving N

In [82]:
BART_BASE_URL = "http://bart.symbotic.corp/"
API_KEY = "E85YQHXLhruypCxny1C6cg"
#databackup = result_df

def requestIonLog(bot_id, startTime, endTime, site_id):

    BART_REQUEST_URL = BART_BASE_URL + "api/v2/retrieve/ionlog_daterange"
    params = {
    "site_id": str(site_id)
    }

    payload = {
        "bot_id": str(bot_id),
        "start_datetime": str(startTime),
        "end_datetime": str(endTime),
        "random_sample": False,
        "max_logs": 100,
        "destination": "../upload",
        "nfs_root": "/botlogs/ionlogs"
    }

    headers ={      
        'Content-Type': 'application/json',
        'api-key': 'E85YQHXLhruypCxny1C6cg'}

    response = requests.post(BART_REQUEST_URL, headers=headers, params=params, json=payload)
    #response.raise_for_status()
    #print(response.status_code)
    response_dict = json.loads(response.text)
    return response_dict

def requestCasLogs(startTime,endTime, site_id, level ):
    BART_REQUEST_URL = BART_BASE_URL + "api/v2/retrieve/cas_daterange"
    params = {
    "site_id": str(site_id)
    }

    payload = {
    "levels": [
        str(level)
    ],
    "start_datetime": str(startTime),
    "end_datetime": str(endTime),
    "max_logs": 100,
    "destination": "../upload",
    "nfs_root": "/logs",
    "cas_folder": "CAS/"
    }

    headers ={      
        'Content-Type': 'application/json',
        'api-key': 'E85YQHXLhruypCxny1C6cg'}

    response = requests.post(BART_REQUEST_URL, headers=headers, params=params, json=payload)
    #response.raise_for_status()
    #print(response.status_code)
    response_dict = json.loads(response.text)
    return response_dict

def requestRouting(startTime, endTime, site_id, level):
    BART_REQUEST_URL = BART_BASE_URL + "api/v2/retrieve/routing_daterange"
    params = {
    "site_id": str(site_id)
    }

    payload = {
    "levels": [
        str(level)
    ],
    "start_datetime": str(startTime),
    "end_datetime": str(endTime),
    "max_logs": 100,
    "subtypes": "all",
    "prefix": "",
    "destination": "../upload",
    "nfs_root": "/logs",
    "routing_folder": "routing/"
    }


    headers ={      
        'Content-Type': 'application/json',
        'api-key': 'E85YQHXLhruypCxny1C6cg'}

    response = requests.post(BART_REQUEST_URL, headers=headers, params=params, json=payload)
    #response.raise_for_status()
    #print(response.status_code)
    response_dict = json.loads(response.text)
    return response_dict
        
def requestVideo():
    return()

def requestInfo(requestID):
    BART_REQUEST_URL = BART_BASE_URL + "api/v2/requests/"
    
    params = {
    "request_id": str(requestID)
    }

    headers ={      
        'Content-Type': 'application/json',
        'api-key': 'E85YQHXLhruypCxny1C6cg'}

    BART_REQUEST_URL = BART_REQUEST_URL + str(requestID)

    response = requests.get(BART_REQUEST_URL, headers=headers, params=params )
    response_dict = json.loads(response.text)
    return response_dict

def downloadRequests(requestIDs,requestName):
    dest_dir = rf'C:\Users\nsegale\OneDrive - Symbotic\Impact_Data\{requestName}'
    os.makedirs(dest_dir, exist_ok=True)
    BART_REQUEST_URL = BART_BASE_URL + fr'/api/v2/download/v2-walmart-tx-newbraunfels-site1/requests/{requestIDs}/walmart-tx-newbraunfels-site1/archive.tar.gz'
    print(BART_REQUEST_URL)
    response = requests.get(BART_REQUEST_URL)
    with open(os.path.join(dest_dir, (requestName +'.tar.gz')), "wb") as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)    
 
    return()

def requestAll(startDate,endDate,botId, siteID, level):
    botId = str(botId)
    site_id = str(siteID)
    startDate = str(startDate)
    endDate = str(endDate)
    level = str(level)
    BART_BASE_URL = "http://bart.symbotic.corp/"
    BART_REQUEST_URL = BART_BASE_URL + "api/v2/retrieve/daterange"
    payload ={
    "bot_id": botId,
        "site_id": siteID,
        "levels": [
        level
        ],
        "start_datetime": startDate,
        "end_datetime": endDate,
        "bot_domain": ".mbot.wmt06016-a.symbotic",
        "artifacts": [
        "ROUTING_DATERANGE",
        "CAS_DATERANGE",
        "IONLOG_DATERANGE",
        "BOT_VIDEOS"
        ],
    "plugins":{
        "plugins.nfs_ionlog_daterange": {
            "bot_id": botId,
            "start_datetime": startDate,
            "end_datetime": endDate,
            "random_sample": False,
            "max_logs": 100,
            "destination": "../upload",
            "nfs_root": "/botlogs/ionlogs"
        },
        "plugins.ssh_bot_videos": {
            "bot_id": botId,
            "start_datetime": startDate,
            "end_datetime": endDate,
            "destination": "../upload",
            "bot_domain": ".mbot.wmt06016-a.symbotic",
            "ssh_user": "botuser",
            "ssh_port": "10022",
            "rsync_bwlimit": None,
            "camera_ids": [
            "3",
            "4",
            "0",
            "6",
            "7",
            "1",
            "5",
            "2"
            ],
            "awk_expr":"", 
            "video_regex":"" 
        },
        "plugins.nfs_cas_daterange": {
            "levels": [
            level
            ],
            "start_datetime": startDate,
            "end_datetime": endDate,
            "max_logs": 100,
            "destination": "../upload",
            "nfs_root": "/logs",
            "cas_folder": "CAS/"
        },
        "plugins.nfs_routing_daterange": {
            "levels": [
            "7"
            ],
            "start_datetime": startDate,
            "end_datetime": endDate,
            "max_logs": 100,
            "subtypes": "all",
            "prefix": "(executive-level|level)",
            "destination": "../upload",
            "nfs_root": "/logs",
            "routing_folder": "routing/"
        }
        },
        "camera_ids": [
        "3",
        "4",
        "0",
        "6",
        "7",
        "1",
        "5",
        "2"
        ],
    "filenames": [],
    "video_endtime_offset": 0,
    "ionlog_starttime_offset": 0,
    "best_effort": True
    }
    headers ={      
        'Content-Type': 'application/json',
        'api-key': 'E85YQHXLhruypCxny1C6cg'}

    response = requests.post(BART_REQUEST_URL, headers=headers, json=payload)
    #response.raise_for_status()
    #print(response.status_code)
    response_dict = json.loads(response.text)
    return response_dict
    

In [84]:
## Make Bart Requests 
for index, row in pending_bart.iterrows():
    if ("BART_ID" in row and pd.notna(pending_bart.at[index, 'BART_ID'])):
        continue 
    else:
        time.sleep(1)
        print(f"Row {index}: Bot ID = {row['BOT_ID']}, Site = {row['SITE_ID']}")
        # Gather Needed Data 
        datetime_loc = pd.to_datetime(row['LOCAL_DATETIME'])
        datetime_utc = pd.to_datetime(row['UTC_DATETIME'])
        startTime = datetime_utc+timedelta(minutes=-3)
        startTime = startTime.replace(microsecond=0)
        try:
            startTime = datetime.strptime(str(startTime), '%Y-%m-%d %H:%M:%S')
        except Exception as e:
            print(e)
        startTime = str(startTime).replace(' ', 'T')
        endTime =  datetime_utc+timedelta(minutes=2)
        endTime = endTime.replace(microsecond=0)
        endTime = datetime.strptime(str(endTime), '%Y-%m-%d %H:%M:%S')
        endTime = str(endTime).replace(' ', 'T')
        site_id = row['SITE_ID']
        bot_id = row['BOT_ID']
        level = row['LOCATION_LEVEL']
        print

        requestRes = requestAll(startTime,endTime,bot_id,site_id,level)
        id = requestRes['id']
        pending_bart.at[index, 'BART_ID'] = id
completed = pd.DataFrame()
check = 1
#loop through requests and download to appropriate locations 
while len(pending_bart) > 0:
    for index, row in pending_bart.iterrows():
        reqId = int(row['BART_ID'])
        status = requestInfo(reqId)['status']
        print("RequestId: " + str(reqId))
        print("Status: " + status)
        if status == "COMPLETED" or status == "PARTIALLY_COMPLETED":
            name = str(row['FILENAME'])
            downloadRequests(reqId,name)
            #completed = pd.concat([completed, result_df.iloc[[index]]], ignore_index=True)
            pending_bart.drop(index, inplace=True)
        elif status == "NO_DATA_FOUND" or status == "FAILED":
            #completed = pd.concat([completed, result_df.iloc[[index]]], ignore_index=True)
            pending_bart.drop(index, inplace=True)
    print("check:" + str(check))
    check +=1
    time.sleep(20)



Row 255: Bot ID = 22314, Site = walmart-tx-newbraunfels-site1
Row 256: Bot ID = 9749, Site = walmart-tx-newbraunfels-site1
Row 257: Bot ID = 31753, Site = walmart-tx-newbraunfels-site1
Row 258: Bot ID = 24525, Site = walmart-tx-newbraunfels-site1
Row 259: Bot ID = 8255, Site = walmart-tx-newbraunfels-site1
Row 260: Bot ID = 22135, Site = walmart-tx-newbraunfels-site1
Row 261: Bot ID = 8255, Site = walmart-tx-newbraunfels-site1
Row 262: Bot ID = 8255, Site = walmart-tx-newbraunfels-site1
Row 263: Bot ID = 8751, Site = walmart-tx-newbraunfels-site1
Row 264: Bot ID = 22163, Site = walmart-tx-newbraunfels-site1
Row 265: Bot ID = 8321, Site = walmart-tx-newbraunfels-site1
Row 266: Bot ID = 31684, Site = walmart-tx-newbraunfels-site1
Row 267: Bot ID = 8577, Site = walmart-tx-newbraunfels-site1
Row 268: Bot ID = 22281, Site = walmart-tx-newbraunfels-site1
Row 269: Bot ID = 31475, Site = walmart-tx-newbraunfels-site1
Row 270: Bot ID = 23836, Site = walmart-tx-newbraunfels-site1
Row 271: Bot ID

KeyboardInterrupt: 